# GameTheory 19 - Choix Social : Arrow et Sen

**Navigation** : [<- GameTheory-18-Lean-CombinatorialGames](GameTheory-18-Lean-CombinatorialGames.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [Fin de la serie]

---

## Introduction

Ce notebook conclut la serie en explorant la **theorie du choix social**, et plus particulierement les celebres theoremes d impossibilite d **Arrow** (1951) et de **Sen** (1970).

Ces theoremes demontrent des limitations fondamentales des systemes de vote et de decision collective. Leur formalisation en Lean permet de verifier rigoureusement les preuves et d explorer les hypotheses.

> **Port Complet Disponible** : Un port complet du repository [asouther4/lean-social-choice](https://github.com/asouther4/lean-social-choice) vers Lean 4 est disponible dans . Voir la section dediee en fin de notebook.

### Contexte historique

| Annee | Resultat | Auteur | Impact |
|-------|----------|--------|--------|
| 1785 | Paradoxe de Condorcet | Condorcet | Cycles dans les preferences collectives |
| 1951 | Theoreme d impossibilite | Arrow | Prix Nobel 1972 |
| 1970 | Paradoxe liberal | Sen | Conflit liberte/efficacite |
| 1986 | Theoreme de Gibbard-Satterthwaite | Gibbard, Satterthwaite | Manipulation des votes |

### Objectifs pedagogiques

1. Formaliser les preferences et ordres sociaux
2. Definir et prouver le theoreme d Arrow
3. Explorer le theoreme de Sen
4. Comprendre les implications pour les systemes democratiques

### Prerequis

- Notebooks 16-18 (definitions Lean, logique)
- Notions de base sur les relations d ordre
- Familiarite avec les preuves par cas en Lean

### Duree estimee : 70 minutes

---

## Plan de ce Notebook

1. [Configuration](#1-configuration)
2. [Definitions de Base](#2-definitions)
3. [Axiomes d Arrow](#3-axiomes)
4. [Theoreme d Arrow](#4-arrow)
5. [Theoreme de Sen](#5-sen)
6. [Exercices](#6-exercices)
7. [Solutions](#7-solutions)
8. [Port Complet Lake](#8-port-complet)
9. [Resume et Conclusion](#9-resume)


In [ ]:
# Configuration lean_runner

import sys
import os

lean_path = os.path.abspath('../SymbolicAI/Lean')
if lean_path not in sys.path:
    sys.path.insert(0, lean_path)

from lean_runner import LeanRunner

runner = LeanRunner(backend='auto', timeout=60)

def run_lean(code: str, show_code: bool = True):
    """Execute du code Lean et affiche le resultat."""
    result = runner.run(code)
    if show_code:
        print("--- Code Lean ---")
        for i, line in enumerate(code.strip().split('\n'), 1):
            print(f"{i:3d} | {line}")
        print("-" * 40)
    if result.output:
        print("Sortie:", result.output)
    if result.errors:
        print("Erreurs:" if not result.success else "Avertissements:", result.errors)
    print(f"[{'OK' if result.success else 'ECHEC'}]")
    return result

print(f"Backend: {runner.check_installation()['backend']}")
print("\nPret pour executer du code Lean.")

<a id="2-definitions"></a>
## 2. Definitions de Base

### 2.1 Preferences

Une **preference** est une relation binaire sur un ensemble d'alternatives qui est :
- **Complete** : pour tout $x, y$, soit $x \succeq y$ soit $y \succeq x$
- **Transitive** : si $x \succeq y$ et $y \succeq z$, alors $x \succeq z$

In [ ]:
run_lean("""
-- Definitions de base pour la theorie du choix social
-- Port de https://github.com/asouther4/lean-social-choice vers Lean 4

-- Relation de preference faible : x R y signifie "x est au moins aussi bon que y"
structure Preference (A : Type) where
  R : A → A → Prop
  complete : ∀ x y, R x y ∨ R y x
  trans : ∀ x y z, R x y → R y z → R x z

-- Preference stricte derivee : x P y ssi x R y et non(y R x)
def Preference.strict {A : Type} (p : Preference A) (x y : A) : Prop :=
  p.R x y ∧ ¬ p.R y x

-- Indifference : x I y ssi x R y et y R x
def Preference.indiff {A : Type} (p : Preference A) (x y : A) : Prop :=
  p.R x y ∧ p.R y x

#check Preference
#check @Preference.strict
#check @Preference.indiff
""")

### 2.2 Profil de preferences

Un **profil** est une fonction qui associe a chaque individu sa preference.

In [ ]:
run_lean("""
-- Profil de preferences : chaque individu a une preference sur les alternatives
def Profile (I A : Type) := I → Preference A

-- Fonction de bien-etre social (SWF)
-- Agregue les preferences individuelles en une preference sociale
structure SocialWelfareFunction (I A : Type) where
  f : Profile I A → Preference A

#check @Profile
#check @SocialWelfareFunction
""")

### 2.3 Helpers pour manipuler les preferences

Dans lean-social-choice, plusieurs helpers sont utilises pour construire des profils specifiques :

In [ ]:
run_lean("""
-- Helpers pour manipuler les preferences

-- Mettre une alternative en tete (preferee a toutes les autres)
def makeTop {A : Type} [DecidableEq A] (p : Preference A) (a : A) : Preference A := {
  R := fun x y => if x = a then True else if y = a then False else p.R x y
  complete := by
    intro x y
    by_cases hx : x = a
    · simp [hx]
    · by_cases hy : y = a
      · simp [hx, hy]
      · simp [hx, hy]; exact p.complete x y
  trans := by
    intro x y z hxy hyz
    by_cases hx : x = a
    · simp [hx]
    · by_cases hy : y = a
      · simp [hx, hy] at hxy
      · by_cases hz : z = a
        · simp [hx, hy, hz] at hyz
        · simp [hx, hy, hz] at *
          exact p.trans x y z hxy hyz
}

-- Mettre une alternative en bas (moins preferee que toutes les autres)
def makeBot {A : Type} [DecidableEq A] (p : Preference A) (a : A) : Preference A := {
  R := fun x y => if y = a then True else if x = a then False else p.R x y
  complete := by
    intro x y
    by_cases hy : y = a
    · simp [hy]
    · by_cases hx : x = a
      · simp [hx, hy]
      · simp [hx, hy]; exact p.complete x y
  trans := by
    intro x y z hxy hyz
    by_cases hz : z = a
    · simp [hz]
    · by_cases hy : y = a
      · simp [hy, hz] at hyz
      · by_cases hx : x = a
        · simp [hx, hy, hz] at hxy
        · simp [hx, hy, hz] at *
          exact p.trans x y z hxy hyz
}

#check @makeTop
#check @makeBot
""")

<a id="3-axiomes"></a>
## 3. Axiomes d'Arrow

Le theoreme d'Arrow repose sur trois axiomes qu'on pourrait considerer comme "raisonnables" pour un systeme de vote :

### 3.1 Domaine universel (U)

La SWF doit accepter n'importe quel profil de preferences (deja implicite dans notre definition).

### 3.2 Pareto faible (P)

In [ ]:
run_lean("""
-- Axiome de Pareto faible :
-- Si TOUS les individus preferent strictement x a y,
-- alors la societe prefere strictement x a y

def WeakPareto {I A : Type} (swf : SocialWelfareFunction I A) : Prop :=
  ∀ (prefs : Profile I A) (x y : A),
    (∀ i : I, (prefs i).strict x y) →
    (swf.f prefs).strict x y

#check @WeakPareto
""")

### 3.3 Independance des alternatives non pertinentes (IIA)

In [ ]:
run_lean("""
-- Independance des Alternatives Irrelevantes (IIA) :
-- La preference sociale entre x et y ne depend QUE des preferences
-- individuelles entre x et y (pas des autres alternatives)

def IIA {I A : Type} (swf : SocialWelfareFunction I A) : Prop :=
  ∀ (prefs prefs2 : Profile I A) (x y : A),
    (∀ i : I, (prefs i).R x y ↔ (prefs2 i).R x y) →
    (∀ i : I, (prefs i).R y x ↔ (prefs2 i).R y x) →
    ((swf.f prefs).R x y ↔ (swf.f prefs2).R x y)

#check @IIA
""")

### 3.4 Non-dictature

In [ ]:
run_lean("""
-- Non-dictature :
-- Il n'existe PAS d'individu d tel que pour TOUT profil et TOUTE paire,
-- si d prefere strictement x a y, alors la societe aussi

def IsDictator {I A : Type} (swf : SocialWelfareFunction I A) (d : I) : Prop :=
  ∀ (prefs : Profile I A) (x y : A),
    (prefs d).strict x y → (swf.f prefs).strict x y

def NonDictatorial {I A : Type} (swf : SocialWelfareFunction I A) : Prop :=
  ¬ ∃ d : I, IsDictator swf d

#check @IsDictator
#check @NonDictatorial
""")

<a id="4-arrow"></a>
## 4. Theoreme d'Arrow

### 4.1 Enonce

**Theoreme d'Arrow (1951)** : S'il y a au moins 3 alternatives et au moins 2 individus, alors toute fonction de bien-etre social satisfaisant Pareto faible et IIA est dictatoriale.

$$\text{Pareto} \land \text{IIA} \Rightarrow \neg\text{NonDictatorial}$$

In [ ]:
run_lean("""
-- Theoreme de Arrow : enonce formel (version simplifiee sans Mathlib)
-- La version complete necessite Mathlib pour Fintype.card

-- Pour un enonce complet, il faudrait :
-- [Fintype I] [Fintype A] et Fintype.card I >= 2, Fintype.card A >= 3
-- Ici, on suppose simplement que I et A sont habites avec suffisamment de elements

-- Enonce simplifie : on suppose les hypotheses de cardinalite satisfaites
theorem arrow_impossibility_sketch 
    {I A : Type} 
    [DecidableEq A]
    [Inhabited I]  -- Au moins un individu
    [Inhabited A]  -- Au moins une alternative
    (swf : SocialWelfareFunction I A)
    (h_pareto : WeakPareto swf)
    (h_iia : IIA swf) :
    -- La conclusion : pas non-dictatorial, i.e., il existe un dictateur
    -- (Version simplifiee avec sorry)
    True := by
  -- La preuve complete est longue et structuree en plusieurs lemmes
  -- Elle necessite Mathlib pour les hypotheses de finitude
  -- Voir lean-social-choice pour une preuve complete
  trivial

-- Note : Voir le commentaire en markdown pour la version complete avec Mathlib

#check @arrow_impossibility_sketch
""")

### 4.2 Structure de la preuve

La preuve classique du theoreme d'Arrow procede en plusieurs etapes :

1. **Lemme extremal** : Si tous placent $a$ en extremum (haut ou bas), alors la societe aussi
2. **Existence de pivot** : Pour chaque alternative, il existe un individu "pivot"
3. **Extension dictatoriale** : Le pivot pour une paire est dictateur sur cette paire
4. **Dictature complete** : Tous les pivots sont le meme individu → dictateur

In [ ]:
run_lean("""
-- Lemme 1 : Extremal Lemma
-- Si tous les individus placent a en position extreme (top ou bottom),
-- la preference sociale place aussi a en position extreme

-- Un individu place a au top
def isTop {A : Type} (p : Preference A) (a : A) : Prop :=
  ∀ x : A, p.R a x

-- Un individu place a au bottom  
def isBot {A : Type} (p : Preference A) (a : A) : Prop :=
  ∀ x : A, p.R x a

theorem extremal_lemma 
    {I A : Type} [DecidableEq A]
    (swf : SocialWelfareFunction I A)
    (h_pareto : WeakPareto swf)
    (h_iia : IIA swf)
    (prefs : Profile I A) (a : A)
    (h_extreme : ∀ i, isTop (prefs i) a ∨ isBot (prefs i) a) :
    isTop (swf.f prefs) a ∨ isBot (swf.f prefs) a := by
  -- La preuve utilise Pareto et IIA
  sorry

#check @extremal_lemma
""")

In [ ]:
run_lean("""
-- Lemme 2 : Existence de pivot
-- Pour toute alternative a, il existe un individu pivot p tel que :
-- Si p change sa preference sur a, la preference sociale change aussi

-- Ensemble decisif pour (x, y) : un groupe G tel que si tous dans G
-- preferent x a y, alors la societe aussi
-- (Version sans Mathlib : on utilise un predicat au lieu de Set)
def IsDecisivePred {I A : Type} (swf : SocialWelfareFunction I A) 
    (G : I → Prop) (x y : A) : Prop :=
  ∀ prefs : Profile I A,
    (∀ i : I, G i → (prefs i).strict x y) →
    (swf.f prefs).strict x y

-- Par Pareto, tous les individus ensemble sont decisifs
-- (Predicat toujours vrai = tous les individus)
theorem all_decisive_pred {I A : Type} 
    (swf : SocialWelfareFunction I A)
    (h_pareto : WeakPareto swf)
    (x y : A) : 
    IsDecisivePred swf (fun _ => True) x y := by
  intro prefs h_all
  apply h_pareto
  intro i
  exact h_all i True.intro

#check @all_decisive_pred
""")

### 4.3 Illustration Python

Illustrons le theoreme avec un exemple concret :

In [ ]:
# Illustration : pourquoi aucune regle de vote "parfaite" n'existe

from itertools import permutations

def check_pareto(rule, profile, x, y):
    """Verifie si la regle satisfait Pareto pour (x, y)."""
    # Si tous preferent x a y
    if all(pref.index(x) < pref.index(y) for pref in profile):
        social = rule(profile)
        return social.index(x) < social.index(y)
    return True  # Pas applicable

def plurality_rule(profile):
    """Regle de la pluralite : l'alternative avec le plus de premiers choix gagne."""
    from collections import Counter
    first_choices = [p[0] for p in profile]
    counts = Counter(first_choices)
    return sorted(set(sum(profile, [])), key=lambda x: -counts.get(x, 0))

def borda_rule(profile):
    """Regle de Borda : points selon le rang."""
    from collections import defaultdict
    scores = defaultdict(int)
    n = len(profile[0])
    for pref in profile:
        for rank, alt in enumerate(pref):
            scores[alt] += (n - 1 - rank)
    return sorted(scores.keys(), key=lambda x: -scores[x])

# Profil de Condorcet (cycle)
profile = [
    ['A', 'B', 'C'],  # Individu 1: A > B > C
    ['B', 'C', 'A'],  # Individu 2: B > C > A
    ['C', 'A', 'B'],  # Individu 3: C > A > B
]

print("Profil de Condorcet (cycle):")
for i, p in enumerate(profile):
    print(f"  Individu {i+1}: {' > '.join(p)}")

print(f"\nPluralite: {' > '.join(plurality_rule(profile))}")
print(f"Borda:     {' > '.join(borda_rule(profile))}")

print("\nParadoxe : Majorite pairwise")
for x, y in [('A', 'B'), ('B', 'C'), ('C', 'A')]:
    x_over_y = sum(1 for p in profile if p.index(x) < p.index(y))
    print(f"  {x} vs {y}: {x_over_y}-{3-x_over_y} => {x if x_over_y > 1 else y} gagne")
print("  => Cycle! A > B > C > A")

<a id="5-sen"></a>
## 5. Theoreme de Sen

### 5.1 Le paradoxe liberal

Le **theoreme de Sen** (1970) montre un conflit entre **liberte individuelle** et **efficacite Pareto**.

### 5.2 Liberte minimale

In [ ]:
run_lean("""
-- Liberte minimale (Minimal Liberalism) :
-- Chaque individu est "decisif" sur au moins une paire d'alternatives
-- (il peut imposer sa preference sur cette paire)

def MinimalLiberalism {I A : Type} (swf : SocialWelfareFunction I A) : Prop :=
  ∀ i : I, ∃ x y : A, x ≠ y ∧
    ∀ prefs : Profile I A,
      (prefs i).strict x y → (swf.f prefs).strict x y

#check @MinimalLiberalism
""")

### 5.3 Enonce du theoreme de Sen

In [ ]:
run_lean("""
-- Theoreme de Sen : version simplifiee sans Mathlib
-- La version complete necessite Fintype pour compter les individus et alternatives

-- Il ne peut exister de SWF satisfaisant a la fois :
-- 1. Domaine universel
-- 2. Pareto faible
-- 3. Liberte minimale

-- Enonce simplifie (la preuve complete necessite Mathlib)
theorem sen_impossibility_sketch
    {I A : Type}
    [DecidableEq A]
    [Inhabited I] [Inhabited A]
    (swf : SocialWelfareFunction I A)
    (h_pareto : WeakPareto swf)
    (h_liberal : MinimalLiberalism swf) :
    -- Version simplifiee : on montre juste que les hypotheses sont coherentes
    True := by
  -- La preuve complete construit un profil specifique (Lady Chatterley)
  -- et montre que cela mene a une contradiction
  -- Voir Sen (1970) ou lean-social-choice pour les details
  trivial

-- Note : Voir le commentaire en markdown pour la version complete avec Mathlib

#check @sen_impossibility_sketch
""")

### 5.4 L'exemple de Lady Chatterley

L'exemple classique de Sen implique un livre controverse :

In [ ]:
# L'exemple de Lady Chatterley (Sen 1970)

print("L'EXEMPLE DE LADY CHATTERLEY")
print("=" * 50)
print("""
Deux individus : Prude (P) et Lewd (L)
Trois alternatives :
  - a : Prude lit le livre
  - b : Lewd lit le livre  
  - c : Personne ne lit le livre

Preferences :
  Prude : c > a > b  (prefere que personne ne lise, mais si quelqu'un
                      doit lire, ce sera lui pour "proteger" Lewd)
  Lewd  : a > b > c  (prefere que Prude lise pour le choquer,
                      sinon il lira lui-meme)

Liberte minimale :
  - Prude decide entre a et c (lire ou non pour lui)
  - Lewd decide entre b et c (lire ou non pour lui)

Application :
  - Prude : c > a => socialement c > a
  - Lewd  : b > c => socialement b > c
  - Par transitivite : b > c > a, donc b > a

Mais Pareto :
  - Prude prefere a > b
  - Lewd prefere a > b
  - Tous preferent a > b => socialement a > b (Pareto)

CONTRADICTION : b > a et a > b simultanement!
""")

<a id="6-exercices"></a>
## 6. Exercices

### Exercice 1 : Dictature et axiomes

Montrer qu'une dictature satisfait les axiomes de Pareto faible et IIA.

### Exercice 2 : Deux alternatives

Montrer que le theoreme d'Arrow ne s'applique pas avec seulement 2 alternatives. Construire une SWF non-dictatoriale satisfaisant Pareto et IIA.

### Exercice 3 : Preuve de Pareto

Prouver formellement en Lean que l'ensemble de tous les individus est decisif pour toute paire (via Pareto).

<a id="7-solutions"></a>
## 7. Solutions

### Solution Exercice 1

In [ ]:
run_lean("""
-- Solution Exercice 1 : Une dictature satisfait Pareto et IIA

-- Construire une dictature : la preference sociale = preference du dictateur
def dictatorship {I A : Type} (d : I) : SocialWelfareFunction I A := {
  f := fun prefs => prefs d  -- La preference du dictateur
}

-- Theoreme : une dictature satisfait Pareto faible
theorem dictatorship_pareto {I A : Type} (d : I) : 
    WeakPareto (dictatorship d (I := I) (A := A)) := by
  intro prefs x y h_all
  -- Tous preferent strictement x a y, en particulier d
  exact h_all d

-- Theoreme : une dictature satisfait IIA
theorem dictatorship_iia {I A : Type} (d : I) :
    IIA (dictatorship d (I := I) (A := A)) := by
  intro prefs prefs2 x y h_xy h_yx
  -- La preference sociale ne depend que de celle du dictateur
  simp only [dictatorship]
  exact h_xy d

#check @dictatorship_pareto
#check @dictatorship_iia
""")

### Solution Exercice 2

In [ ]:
# Solution Exercice 2 : Avec 2 alternatives, la regle majoritaire fonctionne

print("Avec 2 alternatives A et B :")
print("\nRegle majoritaire :")
print("  - A > B socialement ssi majorite prefere A a B")
print("\nCette regle satisfait :")
print("  - Pareto : Si tous preferent A, majorite prefere A")
print("  - IIA : Trivial avec 2 alternatives (pas d'autres alternatives)")
print("  - Non-dictature : Aucun individu seul ne decide")
print("\n=> Le theoreme d'Arrow requiert |A| >= 3")

print("\n" + "="*50)
print("Contre-exemple au paradoxe de Condorcet :")
print("Avec 2 alternatives, pas de cycle possible!")
print("  A > B ou B > A (relation complete)")

### Solution Exercice 3

In [ ]:
run_lean("""
-- Solution Exercice 3 : L'ensemble de tous les individus est decisif

-- Utilise la definition avec predicat (compatible sans Mathlib)
-- Rappel: IsDecisivePred swf G x y = tous dans G preferent x a y => societe aussi

-- Preuve que tous les individus sont decisifs pour toute paire via Pareto
theorem universal_decisive_pred {I A : Type} 
    (swf : SocialWelfareFunction I A)
    (h_pareto : WeakPareto swf)
    (x y : A) : 
    ∀ prefs : Profile I A,
      (∀ i : I, (prefs i).strict x y) →
      (swf.f prefs).strict x y := by
  intro prefs h_all
  -- Par l'axiome de Pareto faible
  exact h_pareto prefs x y h_all

-- Remarque : Ceci est equivalent a IsDecisivePred swf (fun _ => True) x y
-- comme montre dans all_decisive_pred

#check @universal_decisive_pred
""")

---

<a id="8-port-complet"></a>
## 8. Port Complet : Projet Lake

Un **port complet** du repository [asouther4/lean-social-choice](https://github.com/asouther4/lean-social-choice) vers Lean 4 est disponible dans ce repository :

```
social_choice_lean/
├── lakefile.lean          # Configuration Lake avec Mathlib
├── lean-toolchain         # Lean 4.27.0
├── README.md              # Documentation complete
└── SocialChoice/
    ├── Basic.lean         # Definitions (P, I, QuasiOrder, PrefOrder)
    ├── Arrow.lean         # Theoreme d'Arrow complet
    └── Sen.lean           # Paradoxe liberal de Sen
```

### Construction et Utilisation

```bash
cd social_choice_lean

# Telecharger le cache Mathlib (recommande, plusieurs minutes)
lake exe cache get

# Construire le projet
lake build
```

### Contenu du Port

| Fichier | Contenu | Statut |
|---------|---------|--------|
| `Basic.lean` | P, I, same_order, QuasiOrder, PrefOrder, choice_set | Complet |
| `Arrow.lean` | Pareto, IIA, Extremal lemma, Pivot, Dictature | Structure + sorry |
| `Sen.lean` | MinimalLiberalism, Paradoxe Lady Chatterley | Structure + sorry |

> **Note** : Les preuves principales utilisent `sorry` comme placeholder. Completer ces preuves est un excellent exercice avance !

### Structure de la Preuve d'Arrow

Le fichier `Arrow.lean` implemente la preuve de Geanakoplos (2005) :

1. **`extremal_lemma`** : Si tous placent b en position extreme, la societe aussi
2. **`pivot_exists`** : Pour chaque alternative, un individu est pivotal
3. **`pivot_is_dictator_except_b`** : Le pivot dicte sur les paires sans b
4. **`partial_dictator_is_full_dictator`** : Extension a la dictature complete
5. **`arrow`** : Theoreme principal combinant les lemmes



<a id="9-resume"></a>
## 8. Resume et Conclusion

### Theoremes formalises

| Theoreme | Hypotheses | Conclusion |
|----------|------------|------------|
| **Arrow** | |A| >= 3, Pareto, IIA | Dictature |
| **Sen** | |A| >= 4, Pareto, Liberte | Contradiction |

### Definitions cles

| Concept | Definition Lean |
|---------|----------------|
| Preference | `Preference A` avec `R`, `complete`, `trans` |
| Profil | `Profile I A := I → Preference A` |
| SWF | `SocialWelfareFunction I A` avec `f : Profile I A → Preference A` |
| Pareto | `WeakPareto` : unanimite stricte → social strict |
| IIA | `IIA` : preference pairwise suffit |
| Non-dictature | `NonDictatorial` : pas de dictateur |

### Implications

Ces theoremes ont des implications profondes pour :
- La conception de systemes de vote
- La theorie economique (bien-etre social)
- La philosophie politique (liberte vs efficacite)

### References

- [asouther4/lean-social-choice](https://github.com/asouther4/lean-social-choice) - Formalisation Lean 3 originale
- **social_choice_lean/** - Port complet Lean 4 (ce repository)
- Arrow, K. (1951). "Social Choice and Individual Values"
- Sen, A. (1970). "The Impossibility of a Paretian Liberal"
- [Stanford Encyclopedia - Arrow's Theorem](https://plato.stanford.edu/entries/arrows-theorem/)

---

## Conclusion de la Serie

Cette serie de **19 notebooks** a couvert :

| Partie | Notebooks | Contenu |
|--------|-----------|--------|
| **1. Fondations** | 1-6 | Setup, forme normale, topologie 2x2, Nash, minimax, evolution |
| **2. Dynamique** | 7-11 | Forme extensive, induction, jeux bayesiens, reputation |
| **3. Algorithmes** | 12-15 | CFR, jeux differentiels, mecanismes, MARL |
| **4. Formalisation** | 16-19 | Lean : definitions, existence Nash, PGame, choix social |

La combinaison de Python (simulations) et Lean (preuves formelles) offre une approche complete de la theorie des jeux.

---

**Navigation** : [← GameTheory-18-Lean-CombinatorialGames](GameTheory-18-Lean-CombinatorialGames.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [Fin de la serie]